In [ ]:
#Import Necessary Libraries 
import os
import fiona
import rasterio
import rasterio.mask
from rasterio.plot import show
from rasterio.warp import calculate_default_transform, reproject, Resampling

import geopandas as gpd
import pandas as pd
import numpy as np 

import sentinelsat
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import datetime
from datetime import date 

import json
import requests
import zipfile

from osgeo import gdal
import glob

import matplotlib
import matplotlib.pyplot as plt
import imageio
import cv2
from copy import copy 
from scipy import misc


In [ ]:
#Defining Boundary for Download 

#Import shapefile, I'm going to try to use the shapefile as the location, as previous attempts with sentinel don't give 
#the right extent, fairfax gets cut off 

#Read current directory to link to files 
current_dir=os.getcwd()
print(current_dir)

#Linking to Shapefile 
shp_path = current_dir + r'\Desktop\SPR 22 GIS\GGS 416 SAT\final_proj_test1\cb_2018_us_county_5m\fairfax_county.shp'
print(shp_path)

#Reading shapefile
shape_file = gpd.read_file(shp_path)
shape_file.plot()
print(shape_file.crs)
#our shapefiles projection is: epsg:4269

#Convert our shapefile to a geojson
gdf = gpd.read_file(shp_path)
shp_json = gpd.GeoSeries([gdf.geometry.iloc[0]]).__geo_interface__
#checking that the geojson was created 
shp_json

#set shapefile to footprint, for sentinel to search using 
footprint = geojson_to_wkt(shp_json)
#check the geojson filetype converted 
footprint


In [ ]:
#Download Data 

api = SentinelAPI('uqazi2', 'GhoulSzn21!')
api

#Setting date range 
daterange= [
     ('20150505','20150905'),
     ('20160505','20160905'),
     ('20170505','20170905'),
     ('20180505','20180905'),
     ('20190505','20190905'),
     ('20200505','20200905'),
     ('20210505','20210905')
]

#For loop for downloading images 
for x in daterange:
    products = api.query(
        footprint,
        date = (x[0],x[1]),
        cloudcoverpercentage=(0,10),
        limit=1)
    api.download_all(products)
    
    #managing where images are downloaded/stored and zipped 
    all_filenames_in_folder = os.listdir()
    filenames_to_unzip = []
    
    #for loop for unzipping images 
    for filename in all_filenames_in_folder:
        if filename.endswith('zip'):
            filenames_to_unzip.append(filename)
        
    #set folder for downloads 
    folder = 'finalproj_img_stor'
    if not os.path.exists(folder):
        os.mkdir(folder) #creates folder named 'finalproj_img_stor'
        
    for filename in filenames_to_unzip:
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall(folder)
        os.remove(filename)

    print('downloaded')


In [ ]:
#2015 Imagery
#Linking To images 

current_dir=os.getcwd()
print(current_dir)

images_location = current_dir + r'\finalproj_img_stor'
print(images_location)

img15 = images_location + r'\S2A_MSIL1C_20150825T160136_N0204_R054_T18STJ_20150825T160132\GRANULE\L1C_T18STJ_A000909_20150825T160132\IMG_DATA'
bands15 = [img15 + '\T18STJ_20150825T160136_B02.jp2', #blue
           img15 + '\T18STJ_20150825T160136_B03.jp2', #green
           img15 + '\T18STJ_20150825T160136_B04.jp2', #red
           img15 + '\T18STJ_20150825T160136_B05.jp2', #nir 
          ]
bands15

src = rasterio.open(bands15[0])
print(src)

#Checking metadata
meta = src.meta
print(meta)

#updating metadata
meta.update(count=len(bands15))
meta.update(driver='GTiff')
print(meta)


In [ ]:
#Write 4 band image out 
with rasterio.open('C:/Users/cfcni/finalproj_img_stor/2015ffx.tif', 'w', **meta) as dst:
    #For some reason I couldn't use the 'images_location' variable as the path
    for id, layer in enumerate(bands15, start=1):
        with rasterio.open(layer) as src:
            dst.write(src.read(1), id)
            print('Image Written!')
            
#https://geobgu.xyz/py/rasterio.html


In [ ]:
#Lets Clip the Raster to a smaller area using Fairfax City's Shapefile

#Read 4-band image back into Python 
ffx2015_4 = rasterio.open('C:/Users/cfcni/finalproj_img_stor/2015ffx.tif')

#Read Shape File 
shape_file = gpd.read_file(r'C:\Users\cfcni\Desktop\SPR 22 GIS\GGS 416 SAT\final_proj_test1\cb_2018_us_county_5m\fairfax_city.shp')

#Check Projections of Image and Shapefile 
print('Shape File Projection: ', shape_file.crs)
print('Image Projection', ffx2015_4.crs)

#Shape File Projection: epsg:4269
#Images Projection: EPSG:32618

#Specify ouptut projection system 
dst_crs = 'epsg:4269'

#I had to place the entire path in otherwise i would get errors about ambiguity, .all() or .any() wouldn't work
input_imagery_file = (r'C:/Users/cfcni/finalproj_img_stor/2015ffx.tif')
#Save location and file name
transformed_imagery_file = (r'C:/Users/cfcni/finalproj_img_stor/transform_2015ffx_city.tif')

#Transformation process
with rasterio.open(input_imagery_file) as imagery:
    transform, width, height = calculate_default_transform(imagery.crs, dst_crs, imagery.width, imagery.height, *imagery.bounds)
    kwargs = imagery.meta.copy()
    kwargs.update({'crs': dst_crs, 'transform': transform, 'width': width, 'height': height})
    with rasterio.open(transformed_imagery_file, 'w', **kwargs) as dst:
        for i in range(1, imagery.count + 1):
            reproject(
                source=rasterio.band(imagery, i),
                destination=rasterio.band(dst, i),
                src_transform=imagery.transform,
                src_crs=imagery.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)
                            
#Replot image after CRS transformation 
tr_imagery = rasterio.open(r'C:/Users/cfcni/finalproj_img_stor/transform_2015ffx_city.tif')
show(tr_imagery)
tr_imagery.crs

 #re-read shape file 
with fiona.open(r'C:\Users\cfcni\Desktop\SPR 22 GIS\GGS 416 SAT\final_proj_test1\cb_2018_us_county_5m\fairfax_city.shp', 'r') as shapefile:
    shapes = [feature['geometry'] for feature in shapefile]

#read transformed image 
with rasterio.open(r'C:/Users/cfcni/finalproj_img_stor/transform_2015ffx_city.tif') as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop = True)
    out_meta = src.meta
    
#save clipped imagery 
out_meta.update({
    'driver':'GTiff',
    'height': out_image.shape[1],
    'width': out_image.shape[2],
    'transform':out_transform})

with rasterio.open(r'C:/Users/cfcni/finalproj_img_stor/clipped_ffx_city_2015.tif', 'w', **out_meta) as dest:
    dest.write(out_image)

print('Image Clipped')

#tutorial https://thinkinfi.com/clip-raster-with-a-shape-file-in-python/


In [ ]:
#Now lets open our new shapefile 
clipped_image = rasterio.open(r'C:/Users/cfcni/finalproj_img_stor/clipped_ffx_city_2015.tif')
show(clipped_image)


In [ ]:
#Calculating NDVI
#need to read image 
clip_r = clipped_image.read()

#Avoid Errors with 'true divide' 
np.seterr(divide='ignore', invalid='ignore')

#NDVI transformation
ndvi = (clip_r[3].astype(float) - clip_r[2].astype(float)) / (clip_r[3] + clip_r[2])

#Get max and Min Values, excluding NAn
print(np.nanmin(ndvi)) 
print(np.nanmax(ndvi))

#our minimum NDVI value is: -0.32510288065843623
#our maximum NDVI value is: 0.6265402843601896 


In [ ]:
#Visualizing NDVI 
show(ndvi, cmap = 'Reds')

from matplotlib import colors

#apply colorscheme for better visualization 
class MidpointNormalize(colors.Normalize):
   
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
       
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return numpy.ma.masked_array(numpy.interp(value, x, y), numpy.isnan(value))


In [ ]:
#Generate histogram
# Define a new figure
fig2 = plt.figure(figsize=(20,10))

# Give this new figure a subplot, which will contain the histogram itself
ax = fig2.add_subplot(111)

# Add a title & (x,y) labels to the plot
plt.title("NDVI Histogram, 2015", fontsize=18, fontweight='bold')
plt.xlabel("NDVI values", fontsize=20)
plt.ylabel("Number of pixels", fontsize=20)


# For the x-axis, we want to count every pixel that is not an empty value
x = ndvi[~np.isnan(ndvi)]
color = 'g'
# call 'hist` with our x-axis, bins, and color details
ax.hist(x,bins=30,color=color,histtype='bar', ec='black')

# Save the generated figure to an external image file
#fig2.savefig("ndvi-histogram.png", dpi=200, bbox_inches='tight', pad_inches=0.5)
plt.show()

In [ ]:
#EXPORTING NDVI IMAGE TO SINGLE BAND RASTER 

#get the metadata of original GeoTIFF:
meta = src.meta
print(meta)

# get the dtype of our NDVI array:
ndvi_dtype = ndvi.dtype
#ndvi_dtype = ndvi.astype('uint16')
print(ndvi_dtype)

#set the source metadata as kwargs we'll use to write the new data:
kwargs = meta

#update the 'dtype' value to match our NDVI array's dtype:
kwargs.update(dtype=ndvi_dtype)

#update the 'count' value since our output will no longer be a 4-band image:
kwargs.update(count=1)

# Finally, use rasterio to write new raster file 'data/ndvi.tif':
with rasterio.open('ndvi_20152.tif', 'w', **kwargs) as dst:
        dst.write(ndvi, 1)
        print('image written')


In [ ]:
#Bring Back in Single Band NDVI Raster 
ndvi_img_o = rasterio.open('ndvi_2015.tif')

#Checking CRS 
#show(ndvi_img15.read(), transform=ndvi_img15.transform)
print(ndvi_img_o.crs)     #EPSG: 4269

ndvi_img15 = ndvi_img_o.read()
ndvi_img15

#Change data type 
ndvi_img15 = np.float32(ndvi_img15)
ndvi_img15

#create empty frame to store vegetation values 
tree_index = np.zeros(ndvi_img15.shape)

#Filter out low and high values 
tree_index[(ndvi_img15 >= 0.25) & (ndvi_img15 < 0.58)] = 1
#tree_index

#Currently tree_index is a list, we'll convert it to a numpy array and then geojson
tree_index15 = np.array(tree_index)
print(tree_index15)

#Convert from float64 to float32 first
tree_index15 = tree_index15.astype('float32')
show(tree_index15)
tree_index15.dtype

#Convert numpy array into geojson 
my_shapes = shapes(tree_index15)

for my_shape in my_shapes:
    print(my_shape)
    break 
    
#Define Folder name for ouput 
output_folder = 'tree_shapes'

#Check if Folder exists 
if not os.path.exists(output_folder):
    os.mkdir(output_folder) # If not, then make the directory
    print('folder created')
    
#Transform raster coordinates into geographic coordinates 
def transform_coordinates(pair):
    """
    This function takes a pair of raster coordinates 
    and returns the geographic coordinates. 
    
    """
    geo_coords = ndvi_img_o.xy(pair[1],pair[0])

    return [geo_coords[0], geo_coords[1]]

#Create Empty Datafram 
output = [] 

for tree_shape in my_shapes:
    coords = tree_shape[0]['coordinates'][0]
    geographic_coords = [transform_coordinates(pair) for pair in coords]
    # Specify a geojson with our transformed coordinates
    output.append({
        'geometry' : {
            'type':'LineString',
            'coordinates': geographic_coords,
            },
        'properties': {},
    }) 

print('Output contains {} shapes'.format(len(output))) #Contains 5682 shape
#output

#Write shapefile out
data_to_write = gpd.GeoDataFrame.from_features(output, crs='epsg:4269')
data_to_write.to_file('tree_shapes/tree_shape15_4.shp')
print('shapefile created')


In [ ]:
#Try to extract NDVI another way for maksing 
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep


#Calculate NDVI
ndvi2 = es.normalized_diff(clip_r[3], clip_r[2])
ep.plot_bands(ndvi2, cmap="RdYlGn", cols=1, vmin = -1, vmax = 1)
ndvi2

min = np.nanmin(ndvi2)
max = np.nanmax(ndvi2)
print(min)
print(max)

#Classify NDVI values 
ndvi2_class_bins = [-np.inf, 0, 0.1, 0.25, 0.5, np.inf]
ndvi_landsat_class = np.digitize(ndvi2, ndvi2_class_bins)

np.unique(ndvi_landsat_class)
nbr_colors = ['gray','y','darkgreen','black']
nbr_cmap = ListedColormap(nbr_colors)

ndvi2_cat_names = ['No Vegetation','Low Vegetation', 'Vegetation']
classes = np.unique(ndvi_landsat_class)
classes = classes.tolist()
classes = classes[0:3]

fig, ax = plt.subplots(figsize=(12, 12))
im = ax.imshow(ndvi_landsat_class, cmap=nbr_cmap)
ndvi2_class_bins[3].count
